<a href="https://colab.research.google.com/github/MohamedAbshar/Bit-and-Build-2025/blob/main/YOLO_Object_Detection_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Import necessary libraries
import kagglehub
from ultralytics import YOLO
import os
import yaml
import shutil

# --- 1. Download the Object Detection Dataset ---
# We are downloading the TACO dataset, which is pre-formatted for YOLO object detection.
# This means it already includes images, label files with bounding boxes, and a .yaml file.
print("Downloading TACO object detection dataset...")
# Note: This is a different dataset specifically for detection.
dataset_path = kagglehub.dataset_download("kneroma/tacotrashdataset")
print(f"Dataset downloaded to: {dataset_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 2.79G/2.79G [01:37<00:00, 30.6MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/kneroma/tacotrashdataset/versions/3


In [10]:
# --- Step 2: Define Paths for Conversion ---
source_data_dir = os.path.join(dataset_path, 'data')
annotations_path = os.path.join(source_data_dir, 'annotations.json')

# Writable directory for our new YOLO-formatted dataset
yolo_dataset_path = os.path.join('/kaggle/working', 'taco_yolo_dataset')

# Define paths for the new structure
images_path = os.path.join(yolo_dataset_path, 'images')
labels_path = os.path.join(yolo_dataset_path, 'labels')
train_images_path = os.path.join(images_path, 'train')
val_images_path = os.path.join(images_path, 'val')
train_labels_path = os.path.join(labels_path, 'train')
val_labels_path = os.path.join(labels_path, 'val')


In [11]:
# --- Step 3: Create the YOLO Directory Structure ---
print("\n--- Step 2: Creating YOLO directory structure ---")
for path in [train_images_path, val_images_path, train_labels_path, val_labels_path]:
    os.makedirs(path, exist_ok=True)


--- Step 2: Creating YOLO directory structure ---


In [12]:
import json
# --- Step 4: Load and Process Annotations ---
print("\n--- Step 3: Loading and processing annotations.json ---")
with open(annotations_path, 'r') as f:
    coco_data = json.load(f)

# Create mappings
images_info = {img['id']: img for img in coco_data['images']}
categories_info = {cat['id']: cat['name'] for cat in coco_data['categories']}
category_map = {cat['id']: i for i, cat in enumerate(coco_data['categories'])}
class_names = [cat['name'] for cat in coco_data['categories']]

# Group annotations by image_id
annotations_by_image = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(ann)


--- Step 3: Loading and processing annotations.json ---


In [13]:
from sklearn.model_selection import train_test_split
# --- Step 5: Split Data and Perform Conversion ---
print("\n--- Step 4: Splitting data and converting to YOLO format ---")
all_image_ids = list(images_info.keys())
train_ids, val_ids = train_test_split(all_image_ids, test_size=0.2, random_state=42)

def convert_and_move(image_ids, split_name):
    image_dir = os.path.join(images_path, split_name)
    label_dir = os.path.join(labels_path, split_name)

    for img_id in image_ids:
        img_data = images_info[img_id]
        file_name = img_data['file_name']
        img_width = img_data['width']
        img_height = img_data['height']

        # Copy image file
        source_img_path = os.path.join(source_data_dir, file_name)
        if os.path.exists(source_img_path):
            shutil.copy(source_img_path, os.path.join(image_dir, os.path.basename(file_name)))

            # Create label file
            label_file_path = os.path.join(label_dir, os.path.splitext(os.path.basename(file_name))[0] + '.txt')
            with open(label_file_path, 'w') as f:
                if img_id in annotations_by_image:
                    for ann in annotations_by_image[img_id]:
                        cat_id = ann['category_id']
                        yolo_cat_id = category_map[cat_id]

                        # Convert COCO bbox [x_min, y_min, width, height] to YOLO
                        x_min, y_min, w, h = ann['bbox']
                        x_center = (x_min + w / 2) / img_width
                        y_center = (y_min + h / 2) / img_height
                        norm_w = w / img_width
                        norm_h = h / img_height

                        f.write(f"{yolo_cat_id} {x_center} {y_center} {norm_w} {norm_h}\n")

print("Converting training data...")
convert_and_move(train_ids, 'train')
print("Converting validation data...")
convert_and_move(val_ids, 'val')


--- Step 4: Splitting data and converting to YOLO format ---
Converting training data...
Converting validation data...


In [14]:
# --- Step 6: Create the data.yaml File ---
print("\n--- Step 5: Creating the data.yaml file ---")
data_yaml_content = {
    'train': train_images_path,
    'val': val_images_path,
    'nc': len(class_names),
    'names': class_names
}

data_yaml_path = os.path.join('/kaggle/working/', 'dataset.yaml')
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)
print(f"data.yaml created at: {data_yaml_path}")


--- Step 5: Creating the data.yaml file ---
data.yaml created at: /kaggle/working/dataset.yaml


In [ ]:
# --- Step 7: Load and Train the YOLO Model ---
print("\n--- Step 6: Loading and Training the Model ---")
model = YOLO('yolo12n.pt')

print("Starting object detection model training...")
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    project='taco_detection_project',
    name='run1'
)

print("\nTraining complete!")


--- Step 6: Loading and Training the Model ---
Starting object detection model training...
Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run1, nbs=64, nms=False, opset=None, optimize=False, optimi

In [ ]:
# --- Step 8: Export the Best Model ---
print("\n--- Step 7: Exporting the best model ---")
best_model_path = os.path.join('taco_detection_project', 'run1', 'weights', 'best.pt')
destination_path = 'best_taco_detector.pt'
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, destination_path)
    print(f"\nModel saved successfully as '{destination_path}'")
else:
    print("Could not find the best model file.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')